In [2]:
import pandas as pd
import numpy as np
import json

In [ ]:

data = []
with open("AskDocs_submissions.csv", "r") as f:
    for _ in range(10000):
        line = f.readline()
        if not line:
            break
        data.append(json.loads(line))

df = pd.DataFrame(data)

print(df.iloc[:, :10].head())



   over_18 thumbnail  score  ups  \
0    False      self      8    8   
1    False      self      1    1   
2    False      self      1    1   
3    False   default      1    1   
4    False   default      2    2   

                                                 url link_flair_text  \
0  http://www.reddit.com/r/AskDocs/comments/1i467...            None   
1  http://www.reddit.com/r/AskDocs/comments/1i8lp...            None   
2  http://www.reddit.com/r/AskDocs/comments/1i8e3...            None   
3  http://www.reddit.com/r/AskDocs/comments/1i8af...            None   
4  http://www.reddit.com/r/AskDocs/comments/1i87i...            None   

       edited                                           selftext subreddit_id  \
0  1373699396  ~~Welcome psychiatrometer to the subreddit! He...     t5_2xtuc   
1       False  Hello guys, I was wondering if you could share...     t5_2xtuc   
2  1373744170  Let's try to steer the conversation away from ...     t5_2xtuc   
3       False  This is leg

In [4]:
df.columns

Index(['over_18', 'thumbnail', 'score', 'ups', 'url', 'link_flair_text',
       'edited', 'selftext', 'subreddit_id', 'id', 'author', 'selftext_html',
       'user_reports', 'author_flair_text', 'downs', 'distinguished', 'domain',
       'subreddit', 'report_reasons', 'created_utc', 'is_self', 'permalink',
       'stickied', 'title', 'gilded', 'secure_media', 'num_comments', 'media',
       'mod_reports', 'link_flair_css_class', 'secure_media_embed',
       'author_flair_css_class', 'media_embed', 'banned_by', 'retrieved_on',
       'from_id', 'from_kind', 'created', 'quarantine', 'saved', 'name',
       'archived', 'from', 'hide_score'],
      dtype='object')

In [5]:
df['selftext'].iloc[1]

"Hello guys, I was wondering if you could share your thoughts on this thing I've been noticing lately.\n\nHow to describe it... it's like a rush of adrenaline, only there are none of the things that you would attribute to it, that is no increased heart rate, alertness of the senses, or a feeling of thrill or excitement. It feels like a sudden burst of physical (not mental) restlessness or something, or like a weakning of the muscles and it can last anywhere from a minute to maybe fifteen minutes. \n\nI've had this every once in a while now and it never really occured to me to ask anyone about it. It's also kind of hard to describe. Would anyone know what I'm talking about?"

In [ ]:
from together import Together

client = Together(api_key="9e870bfd6d4e11e60c4fb89967d54f32cb7320fca599fe247172bf1f3dc5f898")

high_engagement_posts = []
with open("AskDocs_submissions.csv", "r") as f:
    for _ in range(1000):
        line = f.readline()
        if not line:
            break
        post = json.loads(line)

        if post.get("score", 0) > 10:
            high_engagement_posts.append(post)

if not high_engagement_posts:
    print("No high-engagement posts found.")
    exit()

In [8]:

for post in high_engagement_posts[:1]:
    title = post.get("title", "No title")
    body = post.get("selftext", "No content available")
    prompt = f"Based on the following symptoms, list possible medical conditions and recommend the next diagnostic tests or evaluations a doctor might order: Symptoms: {body}. Include brief justifications for each recommendation."


    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )

    print("\n**Reddit Post Summary:**")
    print(response.choices[0].message.content)



**Reddit Post Summary:**
Based on the provided text, it appears that the "symptoms" are not related to a medical condition, but rather a request for medical professionals to join a subreddit community. However, if we were to interpret this as a hypothetical scenario where a patient is presenting with symptoms related to a desire for online community growth and a need for medical expertise, possible "medical conditions" could be:

1. **Social Media Anxiety Disorder**: A condition where individuals experience anxiety or stress related to their online presence or community growth.
2. **Information Seeking Behavior**: A condition where individuals exhibit a strong desire to seek out and share information, particularly in an online setting.
3. **Community Building Disorder**: A condition where individuals experience a strong desire to build and grow online communities.

Recommended diagnostic tests or evaluations:

1. **Online Behavior Assessment**: A questionnaire or survey to assess the 